# Utilizzare Bert2Bert pre-trained per Summarization su cnn/dailymail

In questo Notebook viene utilizzato un modello pre-trained trovato sull'hub di hugging-face e funzionale per il nostro obiettivo di utilizzare il Transformer Bert per il task di summarization.

Nel paper preso in esame e consultabile al seguente link: http://ceur-ws.org/Vol-2481/paper44.pdf , i risultati ottenuti sul task cnn/dailymail con le metriche R1 ed R2 sono rispettivamente 38.96 e 16.25.

Il modello utilizzante Bert ottiene quindi risultati migliori come si evince eseguendo il codice, R1 = 39.93 ed R2 = 18.22.

Inoltre è stato calcolato anche lo score con la metrica Meteor, non c'era un riferimento nel paper analizzato, ma dalle mie ricerche il punteggio di 28.04 ottenuto con Bert2Bert è superiore a quelli ottenuti da modelli precedenti basati su tecnologie diverse dai Transformer

In [ ]:
!pip install datasets==1.0.2
!pip install transformers==4.2.1

import transformers
import datasets

     |████████████████████████████████| 1.8 MB 7.2 MB/s 
     |████████████████████████████████| 243 kB 62.3 MB/s 
     |████████████████████████████████| 1.8 MB 8.4 MB/s 
     |████████████████████████████████| 895 kB 57.1 MB/s 
     |████████████████████████████████| 2.9 MB 60.1 MB/s 


In [ ]:
!pip install rouge_score
!pip install -U nltk

import nltk
nltk.download('wordnet')

     |████████████████████████████████| 1.5 MB 8.4 MB/s 
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer.bos_token = tokenizer.cls_token
tokenizer.eos_token = tokenizer.sep_token

train_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="train")
val_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="validation[:10%]")

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.61k [00:00<?, ?B/s]

Downloading: 0.00B [00:00, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602. Subsequent calls will reuse this data.


Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)


In [ ]:
from transformers import BertTokenizer, EncoderDecoderModel

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = EncoderDecoderModel.from_pretrained("patrickvonplaten/bert2bert_cnn_daily_mail")
model.to("cuda")

test_data = datasets.load_dataset("cnn_dailymail", "3.0.0", split="test")

batch_size = 32  


def generate_summary(batch):
    # Tokenizer will automatically set [BOS] <text> [EOS]
    # cut off at BERT max length 512
    inputs = tokenizer(batch["article"], padding="max_length", truncation=True, max_length=512, return_tensors="pt")
    input_ids = inputs.input_ids.to("cuda")
    attention_mask = inputs.attention_mask.to("cuda")

    outputs = model.generate(input_ids, attention_mask=attention_mask)

    # all special tokens including will be removed
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    batch["pred"] = output_str

    return batch

results = test_data.map(generate_summary, batched=True, batch_size=batch_size, remove_columns=["article"])

pred_str = results["pred"]
label_str = results["highlights"]

Downloading:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/990M [00:00<?, ?B/s]

Reusing dataset cnn_dailymail (/root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/0128610a44e10f25b4af6689441c72af86205282d26399642f7db38fa7535602)
/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


  0%|          | 0/719 [00:00<?, ?ba/s]

In [ ]:
from datasets import load_metric

rouge = datasets.load_metric("rouge")
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge1"])["rouge1"].mid
print(rouge_output)

Downloading:   0%|          | 0.00/1.66k [00:00<?, ?B/s]

Score(precision=0.3861499253764843, recall=0.4371357928528754, fmeasure=0.3993290655068228)


In [ ]:
rouge_output = rouge.compute(predictions=pred_str, references=label_str, rouge_types=["rouge2"])["rouge2"].mid
print(rouge_output)

Score(precision=0.17675207497792284, recall=0.1990203482945054, fmeasure=0.18215824329209357)


In [ ]:
meteor = datasets.load_metric("meteor")
meteor_out = meteor.compute(predictions=pred_str, references=label_str)
print(meteor_out)

Downloading:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

{'meteor': 0.2804667554262495}
